<a href="https://colab.research.google.com/github/01PrathamS/LLM_Finetuning/blob/main/llm_finetuning_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install datasets evaluate accelerate

In [4]:
from datasets import load_dataset

dataset = load_dataset("mteb/tweet_sentiment_extraction")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})

In [8]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(1000))

In [9]:
dataset["train"]["text"][:10], dataset["train"]["label"][:10]

([' I`d have responded, if I were going',
  ' Sooo SAD I will miss you here in San Diego!!!',
  'my boss is bullying me...',
  ' what interview! leave me alone',
  ' Sons of ****, why couldn`t they put them on the releases we already bought',
  'http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth',
  '2am feedings for the baby are fun when he is all smiles and coos',
  'Soooo high',
  ' Both of you',
  ' Journey!? Wow... u just became cooler.  hehe... (is that possible!?)'],
 [1, 0, 0, 0, 0, 1, 2, 1, 1, 2])

In [10]:
small_train_dataset

Dataset({
    features: ['id', 'text', 'label', 'label_text'],
    num_rows: 1000
})

In [13]:
set(small_train_dataset["label"]), set(small_train_dataset["label_text"])

({0, 1, 2}, {'negative', 'neutral', 'positive'})

In [14]:
from collections import Counter

Counter(small_train_dataset["label_text"])

Counter({'neutral': 383, 'positive': 328, 'negative': 289})

In [15]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

train_tokenized_dataset = small_train_dataset.map(tokenize_function, batched=True)
test_tokenized_dataset = small_test_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)
model.config.pad_token_id = tokenizer.pad_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import evaluate
import numpy as np

metrics = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metrics.compute(predictions=predictions, references=labels)

In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "test_trainer",
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 3,
    gradient_accumulation_steps = 4
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_tokenized_dataset,
    eval_dataset = test_tokenized_dataset,
    compute_metrics = compute_metrics,
)

In [22]:
## evaluate model before training --> how it performs

import evaluate

trainer.evaluate()

{'eval_loss': 2.5778968334198,
 'eval_accuracy': 0.283,
 'eval_runtime': 69.972,
 'eval_samples_per_second': 14.291,
 'eval_steps_per_second': 3.573}

In [23]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=186, training_loss=0.7630811301610803, metrics={'train_runtime': 744.2981, 'train_samples_per_second': 4.031, 'train_steps_per_second': 0.25, 'total_flos': 1555252302053376.0, 'train_loss': 0.7630811301610803, 'epoch': 2.976})

In [24]:
trainer.evaluate()

{'eval_loss': 0.7103415131568909,
 'eval_accuracy': 0.713,
 'eval_runtime': 72.7335,
 'eval_samples_per_second': 13.749,
 'eval_steps_per_second': 3.437,
 'epoch': 2.976}

##

## accuracy before training --> 28%
## accucracy after training --> 71%